# Start a MCP Server

In [ ]:
import subprocess
notebook_subprocesses = []

# Global list to keep track of all subprocesses
notebook_subprocesses = []

In [ ]:
# Start Library Server (port 58002)
proc = subprocess.Popen(["python", "library_server.py"])
notebook_subprocesses.append(proc)

# Smolagents MCP Client Example Usage

In [ ]:
#pip install smolagents[mcp]
from smolagents.mcp_client import MCPClient

tools_url_list = [{"url":"http://localhost:58002/sse"}]

def show_tools(url_list=tools_url_list):
    #for url in url_list:
    try:
        mcp_client = MCPClient(url_list)
        tools = mcp_client.get_tools()
        print("------------------------------------")
        for tool in tools:
            print("tool.name: ", tool.name)
            print("tool.description: ", tool.description)
            print("tool.inputs: ",tool.inputs)
            print("tool.output_type: ", tool.output_type)
            print("------------------------------------")
        # use your tools here.
    finally:
        mcp_client.disconnect()

show_tools()

# Start A2A Server

In [ ]:
import os

openai_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# or
openai_key = "your-openai-key"

In [ ]:
# Start a2a Server with literature critic agent
proc = subprocess.Popen(["python", "literature_critic.py", "--url", "http://localhost:52042", "--openai-key", openai_key])
notebook_subprocesses.append(proc)

# Define a2a SmolAgent proxy from MultiStepAgent

In [ ]:
# Introducing a2a agents as proxies for smolagents
# builds one A2AProxy per remote card
from a2a_bootstrap_proxies import A2AProxyBootstrap

agents_url_list = ["http://localhost:52042"]
smolagent_managed_a2a_agents = A2AProxyBootstrap(agents_url_list).load()

In [ ]:
print("------------------------------------")
for agent in smolagent_managed_a2a_agents:
    print("agent.name: ", agent.name)
    print("agent.description: ", agent.description)
    for skill in agent.card.skills:
        print("agent.skill.name: ", skill.name)
        print("agent.skill.description: ", skill.description)
        print("agent.skill.tags: ", skill.tags)
        print("agent.skill.examples: ", skill.examples)
        print("agent.skill.inputModes: ", skill.inputModes)
        print("agent.skill.outputModes: ", skill.outputModes)
    print("------------------------------------")

# The Host

In [ ]:
from smolagents.models import OpenAIServerModel
from smolagents.agents import CodeAgent
from smolagents.monitoring import LogLevel


model = OpenAIServerModel(
    model_id="gpt-4o", #  note that gpt-4o follows often (not always) a stringent and clear solution path without much ado and provides the correct answer. 4.5 acorns out of five.
    api_key=openai_key,
    temperature=0.3,
)

def run_host_query(user_input: str, url_list=tools_url_list):
    with MCPClient(url_list) as tools:
        orchestrator_agent = CodeAgent(
                                tools=tools,
                                model=model, #ollama_model model
                                managed_agents=smolagent_managed_a2a_agents,
                                verbosity_level= LogLevel.DEBUG
                            )
        return orchestrator_agent.run(user_input)

In [ ]:
run_host_query("Give me a summary of the note on conservation of coffee energy, please.")

In [ ]:
run_host_query("Give me a literature critics review of the quantum squrrel report, please.")

In [ ]:
run_host_query("Give me a literature critics review of the document on the conservation of coffee energy, please.")

# Close a2a agent connections

In [ ]:
for agent in smolagent_managed_a2a_agents:
    if hasattr(agent.a2a_client, "aclose"):
        await agent.a2a_client.aclose()

# Stop all subprocesses

In [ ]:
for proc in notebook_subprocesses:
    proc.terminate()  # or proc.kill() for force
notebook_subprocesses.clear()